In [ ]:
from sklearn.model_selection import train_test_split

from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

# from sklearn.svm import SVC, LinearSVC
# from sklearn.linear_model import SGDClassifier, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

import pandas as pd
import tensorflow as tf
from tensorflow import keras as k

from utility import *

In [ ]:
data = load_data()
X, y = create_XY(data)

enc = OneHotEncoder(sparse=False)
y = enc.fit_transform(y)
X_imputed = SimpleImputer().fit_transform(X)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X_imputed, y, test_size=test_size)

# RandomForestClassifier

In [ ]:
forest = RandomForestClassifier(n_estimators=2, random_state=2)
forest = forest.fit(x_train, y_train)


#Forest Model Metrics
print("Forest Classifier")
print("Train Score: ", forest.score(x_train, y_train))
print("Test Score: ", forest.score(x_test, y_test))

In [ ]:
n = 10
m = 10
max_depth = 10
forests = []
grid = [{"n_estimators": list(range(1, n)), "random_state": list(range(0, m)), "max_depth": list(range(1, max_depth))}]
gridSearch = GridSearchCV(RandomForestClassifier(), param_grid=grid, n_jobs=10, return_train_score=True)
gridSearch.fit(x_train, y_train)

In [ ]:
print("Forest Classifiers Best Score: ", gridSearch.best_score_)
print("Forest Classifiers Best Params: ", gridSearch.best_params_)
print("Forest Classifiers Best Params: ", gridSearch.best_estimator_)
print("Forest Classifiers Best estimator train evaluation: ", gridSearch.best_estimator_.score(x_train, y_train))
print("Forest Classifiers Best estimator test evaluation: ", gridSearch.best_estimator_.score(x_test, y_test))

# Neural Network

In [ ]:
x_train_nn, x_test_nn, y_train_nn, y_test_nn = train_test_split(X_imputed, y, shuffle=False)

In [ ]:
nn = k.models.Sequential([
    k.layers.Flatten(),
    k.layers.Dense(41, activation='relu'),
    k.layers.Dense(75, activation='relu'),
    k.layers.Dropout(0.3),
    k.layers.Dense(3, activation='softmax'),
])

In [ ]:
learning_rate=0.01

nn.compile(
    loss='categorical_crossentropy',
    optimizer=k.optimizers.Adam(learning_rate),
    metrics=['accuracy']
)
nn(x_train_nn)
nn.summary()

In [ ]:
epochs=500
batch_size=50
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=6)
nn.fit(x_train_nn, y_train_nn, epochs=epochs, batch_size=batch_size, callbacks=[callback])

In [ ]:
report1(nn, x_train_nn, y_train_nn, "train", enc)
report1(nn, x_test_nn, y_test_nn, "test", enc)